In [84]:
#Code Prototype for 1D use on EXCLAIM Project with multiple sensors, code by Joaquin Matticoli

In [85]:
import numpy as np
import filterpy
from filterpy.kalman import KalmanFilter as kf
from filterpy.kalman import predict
from filterpy.kalman import update
from numpy.random import randn
from filterpy.common import Q_discrete_white_noise
import matplotlib.pyplot as plt
import filterpy.stats as stats
import math
from math import sqrt

In [86]:
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

def print_gh(predict, update, z1, z2):
    predict_template = '{: 7.3f} {: 8.3f}'
    update_template = '{:.3f}\t{:.3f}\t{: 7.3f} {: 7.3f}'

    print(predict_template.format(predict[0], predict[1]),end='\t')
    print(update_template.format(z1, z2, update[0], update[1]))

In [87]:
class PayloadSimulation(object):
    def __init__(self, x0=0, angular_velocity=1,
                 measurement1_var=0.0, measurement2_var = 0.0,
                 process_var=0.0):
        """ x0 : initial orientation
            angular_velocity: (+=counterclockwise, -=clockwise)
            measurement_var: variances in measurement deg^2
            process_var: variance in process (deg/s)^2
        """
        self.x = x0
        self.angular_velocity = angular_velocity
        self.meas1_std = sqrt(measurement1_var)
        self.meas2_std = sqrt(measurement2_var)
        self.process_std = sqrt(process_var)

    def move(self, dt=1.0):
        """Compute new orientation of the payload in dt seconds."""
        dx = self.angular_velocity + abs(randn()*self.process_std)
        self.x += dx * dt

    def sense_position(self):
        """ Returns measurement of new orientation in degrees."""
        measurement = self.x + abs(randn()*self.meas1_std)
        return measurement
    
    def sense_velocity(self):
        """ Returns measurement of new angular velocity in deg/sec."""
        measurement2 = self.angular_velocity + abs(randn()*self.meas2_std)
        return measurement2

    def move_and_sense(self):
        """ Move payload, and return measurement of new orientation in degrees"""
        self.move()
        return self.sense_position()

In [88]:
printcase = 1 # If set to 1 it will print out the results
plotcase = 0 # If set to 1 it will plot results

#-----------USER INPUT STARTS------------------------------

number_of_variables = 2 #number of variables being measured
number_of_sensors = 2 #number of sensors used
process_var = 25. # variance in the payload's movement, idk what number is good for this??
sensor1_var = 16. # variance in the sensor
sensor2_var = 0.5 # variance in second sensor
x = gaussian(0., 100.)  # payload's initial orientation in deg
angular_velocity = 0.5 # angular velocity in deg/s
dt = 1 # time step in seconds
n_iter = 75 # Number of measurements that the sensor will take

#-----------USER INPUT ENDS--------------------------------

process_model = gaussian(angular_velocity*dt, process_var) # displacement to add to x

    #NOTE: sensor variance is in velocity as it measures that so be careful when creating code for that as it's different to other codes
    #NOTE: this code will assume that orientation is always increasing/decreasing and not sinusodial because a more complext Kalman filter is needed for those cases
    
# simulate payload and get measurements
payload = PayloadSimulation(
    x0=x.mean, 
    angular_velocity=process_model.mean, 
    measurement1_var=sensor1_var, 
    measurement2_var=sensor2_var,
    process_var=process_model.var)

# creates list of measurements, NOTE: modify this section if measurements are provided
zs, zs2, ztrue = [], [], []
for _ in range(n_iter):
    zs.append(payload.move_and_sense())
    zs2.append(payload.sense_velocity())
    ztrue.append(payload.x)
finalposition = payload.x

In [89]:
def plotstuff():
    plt.figure()
    plt.plot(zs,'k+',label='Noisy Measurements')
    plt.plot(x1,'r*',label='A Priori Estimate')
    plt.plot(xhat,'b-',label='A Posteri Estimate')
    plt.plot(ztrue,color='g',label='True Value')
    #plt.plot(xhat,'r*',label='prediction')
    plt.legend()
    plt.title('Estimate vs. Iteration Step', fontweight='bold')
    plt.xlabel('Iteration')
    plt.ylabel('Position')
    
    plt.figure()
    valid_iter = range(1,n_iter) # Pminus not valid at step 0
    plt.plot(valid_iter,Pminus[valid_iter],label='A Posteri Error Estimate')
    plt.title('Estimated $\it{\mathbf{a \ priori}}$ Error vs. Iteration Step', fontweight='bold')
    plt.xlabel('Iteration')
    plt.ylabel('$(Position)^2$')
    #plt.setp(plt.gca(),'ylim',[0,.01])
    plt.show()

x1 = np.zeros(n_iter)
#x2 = np.zeros(n_iter)
Pminus = np.zeros(n_iter)
xhat = np.zeros(n_iter)
counter = 0
if printcase == 1:
    print('PREDICT\t\t\tUPDATE')
    print('     x      var\t\t  z1\tz2\t    x      var')

# perform Kalman filter on measurement(s) z
f = kf(dim_x=number_of_variables, dim_z=number_of_sensors)
f.F = np.array([[1., dt], [0., 1.]]) #correct for calculating position from position and velocity
f.H = np.array([[1., 0.], [0., 1.]]) #this is for first sensor measuring position and second one velocity
f.x = np.array([[x.mean], [angular_velocity*dt+randn()]])
f.Q *= np.array([[(dt**3)/3, (dt**2)/2],
                  [(dt**2)/2,  dt      ]]) * 0.02 #arbitrary
f.P = np.diag([x.var,process_var]) #Variance
f.R = np.diag([sensor1_var, sensor2_var])  #Noise in sensors

for i in range(n_iter):    
    f.predict()
    prior = gaussian(f.x_prior[0,0], f.P_prior[0,0])
    f.update([zs[i], zs2[i]])
    x = gaussian(f.x[0,0], f.P[0,0])
    if printcase == 1:
        print_gh(prior, x, zs[i], zs2[i])

if printcase == 1:
    print()
    print('final estimate:           {:10.3f}'.format(x.mean))
    print('actual final orientation: {:10.3f}'.format(payload.x))
if plotcase == 1:
    plotstuff()

PREDICT			UPDATE
     x      var		  z1	z2	    x      var
  0.036  125.007	6.892	2.065	  6.224  13.803
  8.269   14.432	13.298	0.707	 10.236   7.502
 11.659    8.051	26.934	0.747	 16.452   5.265
 17.807    5.789	24.869	1.288	 19.617   4.158
 21.035    4.669	29.016	0.751	 22.537   3.520
 23.896    4.024	38.269	1.102	 26.599   3.120
 28.078    3.619	38.083	1.156	 29.737   2.856
 31.277    3.351	41.557	0.896	 32.742   2.675
 34.290    3.169	51.043	1.273	 36.850   2.549
 38.558    3.041	48.395	0.891	 39.744   2.459
 41.434    2.951	51.150	0.542	 42.428   2.395
 44.046    2.887	53.801	0.704	 45.110   2.349
 46.698    2.841	64.313	0.813	 48.934   2.315
 50.616    2.808	64.955	1.443	 52.572   2.291
 54.396    2.784	74.259	0.596	 56.721   2.273
 58.594    2.767	72.713	1.449	 60.417   2.261
 62.400    2.755	75.225	0.757	 63.706   2.251
 65.653    2.746	74.271	0.814	 66.401   2.245
 68.273    2.740	80.918	1.722	 69.983   2.240
 71.991    2.735	82.437	0.656	 72.900   2.237
 74.822    2.732	92.353	